# This notebook is for work on my (BD) CS480 research project, with the primary goal of Accurately Forecasting Tornado-forming Severe Storm Systems via Satellite Imagery and Machine Learning

## Preprocessing/Feature engineering

### GLM Data Cleaning

### ABI Data Cleaning

Important resource:
- [Preprocessing steps, as well as beginners guide to using ABI/GOES data](https://www.goes-r.gov/downloads/resources/documents/Beginners_Guide_to_GOES-R_Series_Data.pdf

In [1]:
# install packages
%pip install satpy
%pip install rasterio
%pip install pyresample

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# set up dask
import dask
dask.config.set(num_workers=8)

In [17]:
# imports
import os
import satpy as sat
import pandas as pd
import xarray as xr
from glob import glob

cwd = %pwd
print(cwd)

C:\Users\briid\Documents\Research\Tornado-prediction-with-ML


In [30]:
# example of basic satpy workflow
# reader for ABI: 'abi_l1b'
# reader for GLM: 'glm_l2'
#path = '/content/drive/MyDrive/Colab Notebooks/tornado-forecasting/data/ABI/20220403_00/*.nc'
data_path = './Data/GOES-R/4-06/testing-data/region-2/*.nc'
cwd = %pwd
print(cwd + data_path)
scn = sat.Scene(reader='abi_l1b', filenames=glob(data_path))
scn.load(['C07', 'C08', 'C09'], generate=False)
new_scn = scn.resample(resampler='native')

C:\Users\briid\Documents\Research\Tornado-prediction-with-ML./Data/GOES-R/4-06/testing-data/region-2/*.nc


In [31]:
# save resampled dataset(s) in local file system
new_scn.save_datasets(base_dir='tmp')

type: 'numpy.ndarray' object has no attribute 'attrs'

Note to self: [get back to this once finished with AMVs](https://github.com/pytroll/tutorial-satpy-half-day/blob/main/notebooks/02_reading.ipynb)

### OCTANE setup (for generating Atmospheric Motion Vectors)

#### Initialization:

Important notes on OCTANE:
- Remember to place it between the two timesteps from which the AMVs are derived in relation to the timeseries (i.e. time_of_amv = time_of_timestep_2 - time_of_timestep_1)
- Apparently it was intended for use on cloud- and water-vapor-drift motions (perfect for your use case), so make sure to utilize those three bands from the ABI (8: upper-, 9: mid-, and 10: lower-level water vapor) (as well as maybe the cloud-top phase (11))

In [ ]:
import os
import xarray as xr

base_dir = os.getcwd()  # to come back to home folder after

In [ ]:
# check for cuda driver
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
%pip install netcdf4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.5 MB/s eta 0:00:00


In [ ]:
# install netcdf-cxx4 (necessary for OCTANE)
!apt install libnetcdf-c++4-dev
!apt install netcdf-bin netcdf-doc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libnetcdf-c++4-1
Suggested packages:
  netcdf-bin netcdf-doc
The following NEW packages will be installed:
  libnetcdf-c++4-1 libnetcdf-c++4-dev
0 upgraded, 2 newly installed, 0 to remove and 15 not upgraded.
Need to get 171 kB of archives.
After this operation, 1,444 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libnetcdf-c++4-1 amd64 4.3.1-2build1 [74.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 libnetcdf-c++4-dev amd64 4.3.1-2build1 [95.8 kB]
Fetched 171 kB in 1s (276 kB/s)
Selecting previously unselected package libnetcdf-c++4-1.
(Reading database ... 123105 files and directories currently installed.)
Preparing to unpack .../libnetcdf-c++4-1_4.3.1-2build1_amd64.deb ...
Unpacking libnetcdf-c++4-1 (4.3.1-2build1) ...
Selecting previously unselected package lib

In [ ]:
# no longer needed after building it into the OCTANE directory (modified)

# install cuda samples (used by OCTANE)
#os.chdir('/usr/local/cuda-11.8')
#!rm -r cuda-samples
#!git clone https://github.com/NVIDIA/cuda-samples.git # get cuda samples from the git repo
# move files to proper locations
#!mv cuda-samples/bin/* ./bin/
#!mv cuda-samples/Common/* ./common/
#!mv cuda-samples/Samples/* ./samples/
#!mv cuda-samples/* .

In [ ]:
!ncxx4-config --includedir

/usr/include


In [ ]:
# install octane

# change working directory to OCTANE-master to make and build the program
os.chdir('/content/drive/MyDrive/Colab Notebooks/tornado-forecasting/OCTANE-master/src')  # OCTANE home dir

# make OCTANE program
!make

g++ -o ../build/octane ../build/oct_variational_optical_flow.o ../build/oct_util.o ../build/oct_bicubic.o ../build/oct_zoom.o ../build/oct_gaussian.o ../build/oct_normalize_geo.o ../build/oct_fileread.o ../build/oct_filewrite.o ../build/oct_srsal_cuda.o ../build/oct_patch_match_optical_flow.o ../build/oct_binterp.o ../build/oct_navcal_cuda.o ../build/oct_polar_navcal_cuda.o ../build/oct_merc_navcal_cuda.o ../build/oct_pix2uv_cuda.o ../build/oct_optical_flow.o ../build/oct_interp.o ../build/main.o ../build/jCGlink.o -g -O3 -I /usr/include -I /usr/include -I ../include  -L /usr/lib -lnetcdf -L /usr/lib -lnetcdf_c++4 -L/usr/local/cuda-11.8/lib64 -lcudart -lcudadevrt 


#### Now for experimentation:

In [ ]:
# ensure OCTANE is working properly
os.chdir('/content/drive/MyDrive/Colab Notebooks/tornado-forecasting/OCTANE-master/build')
!./octane

Beginning variational dense optical flow...
Optical Flow Toolkit for Atmospheric aNd Earth sciences (OCTANE)
Author: Jason Apke
Contact: jason.apke@colostate.edu
input flags:

-i1 <filename>, -i2 <filename> are the GOES-R file netcdf full paths, i1 is the first image, i2 is the second

-i1cth <filename>, -i2cth <filename> are optional paths to cloud top height netcdfs 

-nncth instead of default bilinear interpolation, remap the CTH grids with nearest neighbor 

-o <directory> writes the file to the designated directory, include slash at the end (default is ./) 

-pd forces OCTANE to return unnavigated pixel displacements 

-srsal has OCTANE return bilinearly smoothed optical flow output (useful for computing cloud-top divergence) 

-Polar use this flag to ingest polar-orthonormal grid images instead of GOES projections (used for sea-ice tracking)

-Merc use this flag to injest mercator grid images instead of GOES projections (definition) 

-ahi (deprecated) use when reading netcdfs co

In [ ]:
# OCTANE confirmed working!

In [ ]:
# test out OCTANE on some .nc files
#!./octane -alpha -lambda -ir -i1 /content/drive/MyDrive/Colab\ Notebooks/tornado-forecasting/data/ABI/20220403_00/OR_ABI-L1b-RadM1-M6C01_G16_s20220930000278_e20220930000335_c20220930000384.nc -i2 /content/drive/MyDrive/Colab\ Notebooks/tornado-forecasting/data/ABI/20220403_00/OR_ABI-L1b-RadM1-M6C01_G16_s20220930001249_e20220930001306_c20220930001340.nc -o /content/drive/MyDrive/Colab\ Notebooks/tornado-forecasting/OCTANE-out/

Beginning variational dense optical flow...
Here are the file names being used: 
File 1 : /content/drive/MyDrive/Colab Notebooks/tornado-forecasting/data/ABI/20220403_00/OR_ABI-L1b-RadM1-M6C01_G16_s20220930000278_e20220930000335_c20220930000384.nc
File 2 : /content/drive/MyDrive/Colab Notebooks/tornado-forecasting/data/ABI/20220403_00/OR_ABI-L1b-RadM1-M6C01_G16_s20220930001249_e20220930001306_c20220930001340.nc
Number of Blocks per SM (there are 20 SMs) 4
/content/drive/MyDrive/Colab Notebooks/tornado-forecasting/OCTANE-out/outfile.nc written
OCTANE completed, exiting


In [ ]:
# generate AMVs for
#!./octane -alpha 5 -lambda 1 -i1 '/content/drive/MyDrive/Colab Notebooks/tornado-forecasting/data/ABI/20220406/testing-data/region-2/OR_ABI-L1b-RadM2-M6C08_G16_s20220961700549_e20220961701007_c20220961701053.nc' -i2 '/content/drive/MyDrive/Colab Notebooks/tornado-forecasting/data/ABI/20220406/testing-data/region-2/OR_ABI-L1b-RadM2-M6C08_G16_s20220961701549_e20220961702007_c20220961702052.nc' -o '/content/drive/MyDrive/Colab Notebooks/tornado-forecasting/OCTANE-out/output1.nc'
# -ic21 '/content/drive/MyDrive/Colab Notebooks/tornado-forecasting/data/ABI/20220406/testing-data/region-2/OR_ABI-L1b-RadM2-M6C09_G16_s20220961700549_e20220961701012_c20220961701057.nc' -ic22 '/content/drive/MyDrive/Colab Notebooks/tornado-forecasting/data/ABI/20220406/testing-data/region-2/OR_ABI-L1b-RadM2-M6C09_G16_s20220961701549_e20220961702012_c20220961702049.nc' -ic31 '/content/drive/MyDrive/Colab Notebooks/tornado-forecasting/data/ABI/20220406/testing-data/region-2/OR_ABI-L1b-RadM2-M6C10_G16_s20220961700549_e20220961701018_c20220961701052.nc' -ic32 '/content/drive/MyDrive/Colab Notebooks/tornado-forecasting/data/ABI/20220406/testing-data/region-2/OR_ABI-L1b-RadM2-M6C10_G16_s20220961701549_e20220961702018_c20220961702052.nc'  -o '/content/drive/MyDrive/Colab Notebooks/tornado-forecasting/OCTANE-out/output.nc'

Beginning variational dense optical flow...
Here are the file names being used: 
File 1 : /content/drive/MyDrive/Colab Notebooks/tornado-forecasting/data/ABI/20220406/testing-data/region-2/OR_ABI-L1b-RadM2-M6C08_G16_s20220961700549*.nc
File 2 : /content/drive/MyDrive/Colab Notebooks/tornado-forecasting/data/ABI/20220406/testing-data/region-2/OR_ABI-L1b-RadM2-M6C08_G16_s20220961701549*.nc
OCT_GOESREAD FAILURE, CHECK THAT ALL VARIABLES AND ATTS EXIST


In [ ]:
# testing
#!path="/content/drive/MyDrive/Colab Notebooks/tornado-forecasting/data/ABI/test_set_3/" && filename="OR_ABI-L1b*" && echo $filename && find "$path" -name "$filename"
!cd /content/drive/MyDrive/Colab\ Notebooks/tornado-forecasting/data/ABI/test_set_3/ && find . -name OR_ABI-L1b-RadM2-M6C08_G16_s20220961822249*.nc

In [ ]:
# generate AMVs using custom script gen_octane.sh

# copy script to base directory
!cp /content/drive/MyDrive/Colab\ Notebooks/tornado-forecasting/gen_octane.sh /content/
os.chdir('/content/')
# allow execution as script
!chmod +x gen_octane.sh

!./gen_octane.sh 08 20220962010549 20220962014549
!./gen_octane.sh 08 20220962014549 20220962018549
!./gen_octane.sh 08 20220962018549 20220962022549

ABI Data file names: 
  /content/drive/MyDrive/Colab Notebooks/tornado-forecasting/data/ABI/test_set_3/./OR_ABI-L1b-RadM2-M6C08_G16_s20220962010549_e20220962011007_c20220962011056.nc OR_ABI-L1b-RadM2-M6C08_G16_s20220962010549*.nc
  /content/drive/MyDrive/Colab Notebooks/tornado-forecasting/data/ABI/test_set_3/./OR_ABI-L1b-RadM2-M6C08_G16_s20220962014549_e20220962015007_c20220962015056.nc
Output filename: /content/OCTANE-out/amvout_08-20220962010549-20220962014549.nc
Beginning variational dense optical flow...
Here are the file names being used: 
File 1 : /content/drive/MyDrive/Colab Notebooks/tornado-forecasting/data/ABI/test_set_3/./OR_ABI-L1b-RadM2-M6C08_G16_s20220962010549_e20220962011007_c20220962011056.nc
File 2 : /content/drive/MyDrive/Colab Notebooks/tornado-forecasting/data/ABI/test_set_3/./OR_ABI-L1b-RadM2-M6C08_G16_s20220962014549_e20220962015007_c20220962015056.nc
No gpus available for use, exiting

ABI Data file names: 
  /content/drive/MyDrive/Colab Notebooks/tornado-forec

In [ ]:
# load OCTANE output file as xarray
octane_out_file_path = '/content/OCTANE-out/amvout_07-20220961700549-20220961701549.ncoutfile.nc'
octane_test_xr = xr.open_dataset(octane_out_file_path)
octane_test_xr

<xarray.Dataset>
Dimensions:                 (x: 500, y: 500)
Coordinates:
  * x                       (x) float64 -0.03693 -0.03688 ... -0.008988
  * y                       (y) float64 0.1103 0.1102 0.1102 ... 0.0824 0.08235
Data variables: (12/13)
    t                       datetime64[ns] ...
    U                       (y, x) float32 ...
    V                       (y, x) float32 ...
    U_raw                   (y, x) float32 ...
    V_raw                   (y, x) float32 ...
    Rad                     (y, x) float64 ...
    ...                      ...
    optical_flow_settings   int32 ...
    planck_fk1              float32 ...
    planck_fk2              float32 ...
    planck_bc1              float32 ...
    planck_bc2              float32 ...
    kappa0                  float32 ...

In [ ]:
os.chdir('/content/')

In [ ]:
# sweet! everything seems to be working just fine.
# now to figure out how I'm gonna implement/visualize all this...
#   maybe see if you can manipulate the output to look enough like "raw" ABI data to be interpreted
#   in the same way via satpy...?

To visualize with SatPy, one idea for doing so could be to simply pair each coordinate (from OCTANE) with its raw ABI data equivalent, and then plot them where said raw ABI data point is on the SatPy output.
- Steps to do so:
 - Figure out how to pair each OCTANE AMV coordinate with its parent raw data coordinate
 - Figure out how to plot on top of said parent coordinate in its SatPy output
  - Get the visual coordinate for each raw ABI data coordinate
  - Plot the AMV on top of it
  - Repeat (or do in parallel) until all AMVs are paired with their parent ABI imagery coordinates

In [ ]:
# checking satpy readers
#sat.available_readers()

### SPOD/mPOD on AMVs



#### MODULO (mPOD):

Current status of mPOD integration: on hiatus, waiting until first prototype of the model is finished

For mPOD, since [MODULO](https://modulo.readthedocs.io/en/latest/intro.html) is currently only designed for Cartesian grid spaces, we should probably run the AMVs through SatPy (if possible) to properly resample the data and all that.
- This may be able to be done by attaching the AMVs as vector points along with each x/y "pixel" from the GOES image data. this way, it would be able to be reorganized so as to be properly aligned with the pixels from which they were devised.
- Either that, or just run the raw ABI data through SatPy, output to .nc, then run the output through OCTANE... but then OCTANE wouldn't be functioning on the form of data intended for its use... hmmm...
- Maybe do the former (above), and resample it based on their flattened coordinates (e.g. aligned with the resampled/realigned ABI 'pixel' data), so as to produce a new version of the AMV data interpolated from the original, but this time in a "Cartesion"-esque coordinate system, prime for use with MODULO (mPOD)?



Important note on processing speeds:
- MODULO implements a technique (designed to save on RAM in processing) which writes to disk as it runs its calculations. This saves on RAM (particularly for lower-end setups), however I felt it was worth noting in case this may ever be applied to higher-end setups, since reading and writing to the disk slows down the speed at which this model can run. If sufficient RAM is available, I believe that modifications can be made with relative ease in order to speed up the process of computation by writing to RAM instead of disk (should that be an available option, depending on hardware).
- Note to self: given enough time spent on this project, come back and write a function to automatically decide whether or not this should be done based on the amount of available RAM (as well as, if possible, considering the necessary amount of RAM to perform the computations without writing to disk).

Idea: only use mPOD on the area of interest (as per GLM data analysis) (or, use mPOD on all areas of potential interest in parallel)
- Basically, it appears mPOD (via MODULO) is fairly computationally expensive...
- One idea: have options to select to use mPOD, SPOD, or both. This way, processing can be optimized as per hardware limitations
- Another idea: partition the AMV data into "segments" or "squares", and parallelize for processing on GPU. This gives both more precision as well as more optimized processing (theoretically, at least).
 - if you do that, have an option/hyperparameter to change the size of the boxes/partitions/segments

In [ ]:
# install MODULO python package
%pip --no-cache-dir install modulo_vki

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.1/74.1 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 111.2 MB/s eta 0:00:00
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 92, in resolve
    result = self._result = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip

#### PySPOD (SPOD):

##### Initial setup:

In [ ]:
# install pyspod and dependencies
%pip install mpi4py # used for parallel processing via MPI
!git clone https://github.com/MathEXLab/PySPOD.git
%pip install ./PySPOD/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.3 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp310-cp310-linux_x86_64.whl size=3365652 sha256=d9ed29cd083b47eaed2060b9109bb708048b6bf802a5c6aa29f6b768016c5de2
  Stored in directory: /root/.cache/pip/wheels/e8/1b/b5/97ec4cfccdde26e0f3590ad6e09a5242d508dff09704ef86c1
Successfully built mpi4py
Cloning into 'PySPOD'...
remote: Enumerating objects: 4808, done.
remote: Counting objects: 100% (395/395), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 4808 (delta 286), reused 317 (delta 222), pack-reused 4413
Receiving objects: 100% (4808/4808), 154.10 MiB | 13.35 MiB/s, done.
Resolving deltas: 100% (2984/2984), done.
Processing ./PySPOD
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend de

In [ ]:
# base imports
import os
import sys
import numpy as np

# get file paths (for future reference)
CWD = os.getcwd()
CFD = os.path.abspath(CWD)  # full path

# pyspod library specific dependencies
from pyspod.spod.standard import Standard as spod_standard
from pyspod.spod.streaming import Streaming as spod_streaming # alternative spod algorithm
import pyspod.spod.utils as utils_spod
import pyspod.utils.weights as utils_weights
import pyspod.utils.errors as utils_errors
import pyspod.utils.io as utils_io
import pyspod.utils.postproc as post

In [ ]:
# initialize mpi4py (MPI) for parallel processing
try:
  from mpi4py import MPI
  comm = MPI.COMM_WORLD # comms channel for intercommunication between threads
  rank = comm.rank
except:
  comm = None
  rank = 0

##### Working with output from OCTANE:

In [ ]:
# combine datasets(?)
# helpful link --> https://stackoverflow.com/questions/71844625/how-to-add-time-dimension-and-create-an-xarray-dataset-data-array-from-a-stack-o

from datetime import datetime
import pandas as pd

# get file locations
data_files_dir = '/content/OCTANE-out/'
data_files = np.array(os.listdir(data_files_dir))  # get number of files in output folder


# initialize datasets
list_ds = []
#datasets = utils_io.read_data(data_files_dir + data_files[0])
for path in data_files:
  #datasets.combine_first(utils_io.read_data(data_files_dir + x))
  ds = utils_io.read_data(data_files_dir + path)

  time = path.split('-')[1].split('_')[0]
  dt = datetime.strptime(time, "%Y%j%H%M%S%f")
  dt = pd.to_datetime(dt)

  ds = ds.assign_coords(time = dt)
  ds = ds.expand_dims(dim = "time")

  list_ds.append(ds)

# stack datasets in list(?)
ds = xr.combine_by_coords(list_ds, combine_attrs="override") # in this case overriding is fine
print(ds)



<xarray.Dataset>
Dimensions:                 (x: 500, y: 500, time: 3)
Coordinates:
  * x                       (x) float64 -0.03693 -0.03688 ... -0.008988
  * y                       (y) float64 0.1103 0.1102 0.1102 ... 0.0824 0.08235
  * time                    (time) datetime64[ns] 2022-04-06T17:00:54.900000 ...
Data variables: (12/13)
    t                       (time) datetime64[ns] 2022-04-06T17:00:58.433936 ...
    U                       (time, y, x) float32 16.77 16.77 ... 44.76 44.76
    V                       (time, y, x) float32 -0.43 -0.43 ... -1.02 -1.02
    U_raw                   (time, y, x) float32 0.44 0.44 0.44 ... 1.3 1.3 1.3
    V_raw                   (time, y, x) float32 0.0 0.0 0.0 ... 0.01 0.01 0.01
    Rad                     (time, y, x) float64 1.025 1.046 ... 0.7368 0.7368
    ...                      ...
    optical_flow_settings   (time) int32 -2147483647 -2147483647 -2147483647
    planck_fk1              (time) float32 2.023e+05 2.023e+05 2.023e+05
  

In [ ]:
# pull what we want to analyze from ds1, ds2, and ds3

t = np.array(ds['time'])
# x1 and x2 (lon/lat) can be the same, since each timestep covers the same coords
x1 = np.array(ds['x'])
x2 = np.array(ds['y'])
nt = t.shape[0]


In [ ]:
# load data to be analyzed (U and V) via PySPOD
variables = ['U', 'V']
X = np.empty([t.shape[0], x1.shape[0], x2.shape[0], len(variables)])
for i, var in enumerate(variables):
  X[...,i] = np.einsum('ijk->ikj', np.array(ds[var])) # basically swapping coordinates (lon/lat and x/y are handled differently)
  X[...,i] = np.nan_to_num(X[..., i])
print('shape of data matrix X: ', X.shape)

shape of data matrix X:  (3, 500, 500, 2)


## MS-LSTM/MS-RNN Implementation:

### Initialization:

Utilizing resources from the [MS-RNN Github repo](https://github.com/mazhf/MS-RNN/tree/main)

Important note: considering the limitations of free Google Colab, I am going to be doing initial experimentation on my own hardware (despite the limitations of not having a NVIDIA GPU currently accessible to me)

In [3]:
# clone github repo
!git clone https://github.com/mazhf/MS-RNN.git

Cloning into 'MS-RNN'...


In [17]:
!pip uninstall h5py

^C


In [18]:
# install libraries, as per the included README
!pip install --use-pep517 -r ./MS-RNN/requirements.txt --extra-index-url https://download.pytorch.org/whl/cu113

^C
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
  Using cached https://download.pytorch.org/whl/cu113/torch-1.12.0%2Bcu113-cp310-cp310-win_amd64.whl (2143.8 MB)
  Using cached numpy-1.23.5-cp310-cp310-win_amd64.whl (14.6 MB)
  Using cached opencv_python-4.8.0.74-cp37-abi3-win_amd64.whl (38.1 MB)
  Using cached h5py-2.9.0.tar.gz (287 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
  Using cached tensorboardX-2.6.1-py2.py3-none-any.whl (101 kB)
  Using cached protobuf-4.23.4-cp310-abi3-win_amd64.whl 

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [1326 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\h5py
  copying h5py\h5py_warnings.py -> build\lib.win-amd64-cpython-310\h5py
  copying h5py\highlevel.py -> build\lib.win-amd64-cpython-310\h5py
  copying h5py\ipy_completer.py -> build\lib.win-amd64-cpython-310\h5py
  copying h5py\version.py -> build\lib.win-amd64-cpython-310\h5py
  copying h5py\__init__.py -> build\lib.win-amd64-cpython-310\h5py
  creating build\lib.win-amd64-cpython-310\h5py\_hl
  copying h5py\_hl\attrs.py -> build\lib.win-amd64-cpython-310\h5py\_hl
  copying h5py\_hl\base.py -> build\lib.win-amd64-cpython-310\h5py\_hl
  copying h5py\_hl\compat.py -> build\lib.win-amd64-cpython-310\h5py\_hl
  copying h5py\_hl\dataset.py -> build\lib.win-amd64-cpython-310\h5py\_hl
  copying h5py\_hl\datatype.py -> build\lib.win-amd64-cpython-31

In [5]:
# installing local attention (only needed for PrecipLSTM/MS-PrecipLSTM):
!cd ./MS-RNN/img_local_att && python setup.py install

Traceback (most recent call last):
  File "C:\Users\briid\Documents\Research\Tornado-prediction-with-ML\experimentation\MS-RNN\img_local_att\setup.py", line 10, in <module>
    assert cuda.is_available(), 'Please install CUDA for GPU support.'
AssertionError: Please install CUDA for GPU support.


In [9]:
!python -m torch.distributed.launch --nproc_per_node=4 ./MS-RNN/main.py

/usr/local/lib/python3.10/dist-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
2.61055242306030742.6105524230603074
2.6105524230603074

2.6105524230603074
Traceback (most recent call last):
  File "/content/./MS-RNN/main.py", line 12, in <module>
    from net_params import nets
  File "/content/MS-RNN/net_params.